# Matrix Computation

## 重复本讲义三对角高斯消元法


In [33]:
import numpy as np

def LUDecompose(A):
    """
    returns L,U, where the diagonal elements of U are all 1.
    """
    # the order of the matrix
    N=np.size(A,0)
    L=np.zeros_like(A)
    U=np.identity(N)
    # Be careful that the index starts from 0.
    L[0,0]=A[-0,0]
    U[0,1]=A[0,1]/A[0,0]
    for i in range(1,N):
        L[i,i-1]=A[i,i-1]
        L[i,i]=A[i,i]-L[i,i-1]*U[i-1,i]
        if i<N-1:
            U[i,i+1]=A[i,i+1]/L[i,i]
    return L,U

In [44]:
def solveLU(L,U,d):
    """
    solve a set of linear equations written in the form LUx=d.
    """
    N=np.size(L,1)
    y=np.zeros_like(d)
    x=np.zeros_like(d)
    y[0]=d[0]/L[0,0]
    for i in range(1,N):
        y[i]=(d[i]-L[i,i-1]*y[i-1])/L[i,i]
    x[N-1]=y[N-1]
    for i in np.arange(N-2,-1,-1):
        x[i]=y[i]-U[i,i+1]*x[i+1]
    return x

def solve(A,d):
    return solveLU(*LUDecompose(A),d) # love the argument unpacking grammar!

Using the examples in the textbook to verify the results.

In [41]:
A=np.array([[ 2,-1, 0, 0],
            [-1, 3, -2,0],
            [ 0,-2, 4,-3],
            [ 0, 0,-3, 5]],dtype=float)
d=np.array([6,1,-2,1],dtype=float).reshape((4,1))

In [43]:
L,U=LUDecompose(A)
print("L:")
print(L)
print("U:")
print(U)

x=solve(A,d)
print("solution x:")
print(x)

L:
[[ 2.    0.    0.    0.  ]
 [-1.    2.5   0.    0.  ]
 [ 0.   -2.    2.4   0.  ]
 [ 0.    0.   -3.    1.25]]
U:
[[ 1.   -0.5   0.    0.  ]
 [ 0.    1.   -0.8   0.  ]
 [ 0.    0.    1.   -1.25]
 [ 0.    0.    0.    1.  ]]
solution x:
[[5.]
 [4.]
 [3.]
 [2.]]


## 利用三种迭代方法求解 P148#13

$$
\begin{bmatrix}
12&-2&1&&&\\
-2&12&-2&1&&\\
1&-2&12&-2&1&\\
&1&-2&12&-2&1\\
&&\cdots\\
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
\vdots\\
x_{49}\\
x_{50}
\end{bmatrix}
=
\begin{bmatrix}
5\\
5\\
5\\
5\\
5\\
\end{bmatrix}
$$

We have chosen to represent and store the coefficient matrix in sparse matrix `csr_matrix` encoding.

In [65]:
from scipy.sparse import csr_matrix
# Note the weird grammar in sum().
rows = np.array([0]*3+[1]*4+sum([[i]*5 for i in range(2,48)],[])+[48]*4+[49]*3)
columns=np.array([0,1,2]+[0,1,2,3]+sum([list(range(i,i+5)) for i in range(46)],[])+[46,47,48,49]+[47,48,49])
data=np.array([12,-2,1]+[-2,12,-2,1]+[1,-2,12,-2,1]*46+[1,-2,12,-2]+[1,-2,12])

sparseA=csr_matrix((data,(rows,columns)),shape=(50,50)).toarray()
print(sparseA)
# print(type(sparseA))
b=np.ones((50,1),dtype=float)*5
print(b)

[[12 -2  1 ...  0  0  0]
 [-2 12 -2 ...  0  0  0]
 [ 1 -2 12 ...  0  0  0]
 ...
 [ 0  0  0 ... 12 -2  1]
 [ 0  0  0 ... -2 12 -2]
 [ 0  0  0 ...  1 -2 12]]
[[5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]
 [5.]]


In [25]:
import scipy.sparse
# yet a more elegant way to generate the sparse matrix
sparseA=scipy.sparse.diags([1,-2,12,-2,1],[-2,-1,-0,1,2],shape=(50,50)).toarray()
print(sparseA)

[[12. -2.  1. ...  0.  0.  0.]
 [-2. 12. -2. ...  0.  0.  0.]
 [ 1. -2. 12. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 12. -2.  1.]
 [ 0.  0.  0. ... -2. 12. -2.]
 [ 0.  0.  0. ...  1. -2. 12.]]


In [61]:
def Jacobi(A,b):
    D_inverse=np.linalg.inv(np.diag(np.diag(A))) # nice function diag!
    M=np.identity(np.size(A,1))-np.dot(D_inverse,A)
    g=np.dot(D_inverse,b)
    return M,g

def Gauss_Seidel(A,b):
    U=np.triu(A,1)
    L=np.tril(A,-1)
    D=np.diag(np.diag(A))
    DL_inverse=np.linalg.inv(D+L)
    M= -np.dot(DL_inverse,U)
    g=np.dot(DL_inverse,b)
    return M,g

def Relaxation(A,b,w=1):
    """
    w=1, i.e. Gauss_Seidel by default
    """
    U=np.triu(A,1)
    L=np.tril(A,-1)
    D=np.diag(np.diag(A))
    DL_inverse=np.linalg.inv(D+w*L)
    M= np.dot(DL_inverse,D-w*(D+U))
    g=np.dot(DL_inverse,b)*w
    return M,g

In [68]:
# Iterative method

def iterativeSolution(A,b,x0,epsilon=1e-6,method=Jacobi):
    """
    solve the linear equations in the form Ax=b, A,b are required to be in array.
    the available methods are: Jacobi, Gauss_Seidel, Relaxation
    """
    # generates the M and g matrix in different methods.
    M,g=method(A,b) 
    x=np.array(x0)
    newX=np.dot(M,x)+g
    #iteration times
    i=1 
    while np.linalg.norm(newX-x)>=epsilon:
        x=newX
        newX=np.dot(M,x)+g
        i+=1
    return newX,i

In [111]:
x0=np.zeros((50,1),dtype=float)
weightList=np.arange(0.8,1.5,0.1)

methodList=[Jacobi,Gauss_Seidel,lambda A,b:Relaxation(A,b,1.2)]
l=[iterativeSolution(sparseA,b,x0,method=each,epsilon=1e-6) for each in methodList]

# print(l)

table=np.hstack([each[0]for each in l])
print("Jacobi\tGS\tRelaxation(w=1.2)")
# times of iteration
for i in l:
    print(i[1],end='\t')
print()
print(table)

Jacobi	GS	Relaxation(w=1.2)
15	9	14	
[[0.46379555 0.46379552 0.46379552]
 [0.53728457 0.5372846  0.53728461]
 [0.50902297 0.50902292 0.50902292]
 [0.49822158 0.49822163 0.49822163]
 [0.49894192 0.49894186 0.49894186]
 [0.49998529 0.49998535 0.49998535]
 [0.50008878 0.50008872 0.50008872]
 [0.50001527 0.50001532 0.50001532]
 [0.49999483 0.49999479 0.49999479]
 [0.49999782 0.49999786 0.49999786]
 [0.50000013 0.50000011 0.50000011]
 [0.50000018 0.5000002  0.5000002 ]
 [0.50000004 0.50000002 0.50000002]
 [0.49999998 0.49999999 0.49999999]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5

As we can see, the result above shows a good agreement with each other. Gauss-Seidel Method performs better than the Jacobi method.
To test the convergence speed, we change the weight in relaxation method and print the table.

In [117]:
print("Method  \titerations")
print("Jacobi  \t{}".format(iterativeSolution(sparseA,b,x0,method=Jacobi)[1]))
print("Gauss-Seidel\t{}".format(iterativeSolution(sparseA,b,x0,method=Gauss_Seidel)[1]))
for i in np.arange(0.1,2.0,0.1):
    print("Relax(w={:.1f})\t{}".format(i,iterativeSolution(sparseA,b,x0,method=lambda A,b:Relaxation(A,b,i))[1]))

Method  	iterations
Jacobi  	15
Gauss-Seidel	9
Relax(w=0.1)	146
Relax(w=0.2)	73
Relax(w=0.3)	48
Relax(w=0.4)	35
Relax(w=0.5)	27
Relax(w=0.6)	21
Relax(w=0.7)	17
Relax(w=0.8)	13
Relax(w=0.9)	11
Relax(w=1.0)	9
Relax(w=1.1)	11
Relax(w=1.2)	14
Relax(w=1.3)	18
Relax(w=1.4)	22
Relax(w=1.5)	29
Relax(w=1.6)	39
Relax(w=1.7)	56
Relax(w=1.8)	80
Relax(w=1.9)	142


In conclusion, we need to adjust the weight in relaxation method carefully to get a good performance. In this case, the best performance coincides with w=1(Gauss-Seidel Method).